# Community Detection in FriendFeed Network

This notebook focuses on detecting communities within the FriendFeed social network. We'll use the preprocessed datasets to build a graph and analyze community structures.

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
from community import community_louvain

## Savle the network

In [ ]:
# Read the graph from the GraphML file
G = nx.read_graphml("graph/graph.graphml")
print(nx.info(G))

## 3. Analyze Network Structure

Let's examine the basic characteristics of the network, such as degree distributions.

In [ ]:
# Calculate degree distributions
degrees = dict(G.degree())
in_degrees = dict(G.in_degree())
out_degrees = dict(G.out_degree())

# Plot degree distribution
plt.figure(figsize=(10, 6))
sns.histplot(list(degrees.values()), bins=30, kde=True)
plt.title('Degree Distribution')
plt.xlabel('Degree')
plt.ylabel('Frequency')
plt.show()

## 4. Detect Communities Using the Louvain Method

The Louvain algorithm is a popular method for community detection in large networks. We'll use this method to detect communities and analyze them.

In [ ]:
%%time
# Convert graph to undirected for community detection
G_undirected = G.to_undirected()

# Apply Louvain method for community detection
partition = community_louvain.best_partition(G_undirected)
communities = set(partition.values())
num_communities = len(communities)

print(f"Number of communities detected: {num_communities}")

# Add community information as node attribute
nx.set_node_attributes(G, partition, 'community')

## 5. Visualize Communities

Let's visualize the detected communities. Each node will be colored based on the community it belongs to.

In [ ]:
plt.figure(figsize=(12, 12))
pos = nx.spring_layout(G_undirected, k=0.1)

# Draw nodes with colors according to community
colors = [partition[node] for node in G_undirected.nodes]
nx.draw_networkx_nodes(G_undirected, pos, node_color=colors, cmap=plt.cm.jet, node_size=20)

# Draw edges
nx.draw_networkx_edges(G_undirected, pos, alpha=0.1)

plt.title('Community Detection using Louvain Method')
plt.axis('off')
plt.show()

## 6. Community Analysis

We can further analyze the communities to understand their sizes and characteristics.

In [ ]:
# Calculate size of each community
community_sizes = pd.Series(list(partition.values())).value_counts()

plt.figure(figsize=(10, 6))
sns.barplot(x=community_sizes.index, y=community_sizes.values, palette='viridis')
plt.title('Community Size Distribution')
plt.xlabel('Community ID')
plt.ylabel('Number of Nodes')
plt.show()

## 7. Save the Results

Finally, let's save the graph with community information for further analysis if needed.

In [ ]:
# Save graph with community info
nx.write_gexf(G, 'friendfeed_community_graph.gexf')
print("Graph with community information saved as 'friendfeed_community_graph.gexf'.")